# インタラクティブに実行する

以下の流れで、COMBO をインタラクティブに実行することができます。

1. COMBO から次に実行するパラメータを得る
2. COMBO の外部で評価値を得る
3. 評価値をCOMBOに登録する

例えば、以下の様な場合に適しています。

- 人手による実験を行い、その評価値をCOMBOに与えたい
- simulator の実行を別プロセスで行うなど、柔軟に実行制御を行いたい

## 探索候補データの準備

In [1]:
import combo

import os
import urllib
import ssl
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ssl._create_default_https_context = ssl._create_unverified_context

def download():
    if not os.path.exists('data/s5-210.csv'):

        if not os.path.exists('data'):
            os.mkdir('data')
            
        print('Downloading...')
        with urllib.request.urlopen("http://www.tsudalab.org/files/s5-210.csv") as response, open('data/s5-210.csv', 'wb') as out_file:
            out_file.write(response.read())
        print('Done')
        
def load_data():
    download()
    A =  np.asarray(np.loadtxt('data/s5-210.csv',skiprows=1,delimiter=',') )
    X = A[:,0:3]
    t  = -A[:,3]
    return X, t

X, t = load_data()
X = combo.misc.centering( X )

## simulator の定義

In [2]:
class simulator:
    def __init__( self ):
        _, self.t = load_data()
    
    def __call__( self, action ):
        return self.t[action]

## 最適化の実行

In [3]:
# policy のセット 
policy = combo.search.discrete.policy(test_X=X)

# シード値のセット 
policy.set_seed( 0 )

各探索ステップでは以下の処理を行っています。

1. `max_num_probes=1, simulator=None` として random_search または bayes_search を実行して actions を得る
2. `t  = simulator(actions)` により評価値(の array) を得る
3. `policy.write(actions, t)`により action に対する評価値を登録する
4. `combo.search.utility.show_search_results` により履歴を表示する

In [4]:
simulator = simulator()

''' 1st step (random sampling) '''
actions = policy.random_search(max_num_probes=1, num_search_each_probe=10, simulator=None)
t  = simulator(actions)
policy.write(actions, t)
combo.search.utility.show_search_results(policy.history, 10)

''' 2nd step (random sampling) '''
actions = policy.random_search(max_num_probes=1, num_search_each_probe=10, simulator=None)
t = simulator(actions)
policy.write(actions, t)
combo.search.utility.show_search_results(policy.history, 10)

''' 3rd step (bayesian optimization) '''
actions = policy.bayes_search(max_num_probes=1, num_search_each_probe=10, 
                                                      simulator=None, score='EI', interval=0,  num_rand_basis = 0)
t = simulator(actions)  
policy.write(actions, t) 
combo.search.utility.show_search_results(policy.history, 10) 

''' 4-th step (bayesian optimization) '''
actions = policy.bayes_search(max_num_probes=1, num_search_each_probe=10, 
                                                      simulator=None, score='EI', interval=0,  num_rand_basis = 50)
t = simulator(actions) 
policy.write(actions, t)
combo.search.utility.show_search_results(policy.history, 10)

interactive mode stars ... 
 
0001-th multiple probe search (random) 

current best f(x) = -0.980054 (best action = 4547) 
list of simulation results
f(x)=-1.070602 (action = 15673)
f(x)=-1.009056 (action = 9559)
f(x)=-1.195844 (action = 16927)
f(x)=-0.980054 (action = 4547)
f(x)=-0.992820 (action = 2553)
f(x)=-1.146676 (action = 13144)
f(x)=-1.006255 (action = 10827)
f(x)=-0.999862 (action = 1995)
f(x)=-1.055445 (action = 10763)
f(x)=-1.100970 (action = 16450)


0002-th multiple probe search (random) 

current best f(x) = -0.980054 (best action = 4547) 
list of simulation results
f(x)=-1.208666 (action = 13085)
f(x)=-1.069404 (action = 15133)
f(x)=-1.031642 (action = 1706)
f(x)=-1.016702 (action = 2464)
f(x)=-1.172569 (action = 17812)
f(x)=-1.082219 (action = 16533)
f(x)=-1.025272 (action = 1336)
f(x)=-1.031761 (action = 10076)
f(x)=-0.984972 (action = 8876)
f(x)=-1.107730 (action = 15577)


Start the initial hyper parameter searching ...
Done

Start the hyper parameter learning ...
0

## 中断と再開

以下の predictor, training, history を外部ファイルに保存することで、最適化プロセスを中断し、途中から再開することができます。

- predictor: 目的関数の予測モデル
- training: predictor の学習に用いるデータ (`combo.variable` オブジェクト）
- history: 最適化実行の履歴 (`combo.search.discrete.results.history` オブジェクト)

In [5]:
import pickle

with open('predictor.dump', 'wb') as f:
    pickle.dump(policy.predictor, f)
policy.training.save('training.npz')
policy.history.save('history.npz')

In [6]:
# policy を削除
del policy

# 保存した policy をロード 
policy = combo.search.discrete.policy(test_X=X)
policy.load('history.npz', 'training.npz', 'predictor.dump')

''' 5-th probe (bayesian optimization) '''
actions = policy.bayes_search(max_num_probes=1, num_search_each_probe=10, 
                                                      simulator=None, score='EI', interval=0,  num_rand_basis = 0)
t = simulator(actions) 
policy.write(actions, t) 
combo.search.utility.show_search_results(policy.history, 10) 

# predictor と training を個別に指定することも可
''' 6-th probe (bayesian optimization) '''
actions = policy.bayes_search(max_num_probes=1, num_search_each_probe=10, 
                                            predictor=policy.predictor, training=policy.training,
                                            simulator=None, score='EI', interval=0,  num_rand_basis = 0)
t = simulator(actions) 
policy.write(actions, t) 
combo.search.utility.show_search_results(policy.history, 10) 

Start the initial hyper parameter searching ...
Done

Start the hyper parameter learning ...
0-th epoch, marginal likelihood -27.8101674631
50-th epoch, marginal likelihood -38.5151806586
100-th epoch, marginal likelihood -44.3860399514
150-th epoch, marginal likelihood -47.687639627
200-th epoch, marginal likelihood -49.6338245999
250-th epoch, marginal likelihood -50.8300879488
300-th epoch, marginal likelihood -51.5973262888
350-th epoch, marginal likelihood -52.1139626467
400-th epoch, marginal likelihood -52.4824699563
450-th epoch, marginal likelihood -52.7627945703
500-th epoch, marginal likelihood -52.9902029595
Done

 Parameters of Gaussian kernel 
 
 width  =  [ 0.65397154]
 scale  =  0.0755729095878
 scale2 =  0.00571126466357
 

0005-th multiple probe search (EI) 

current best f(x) = -0.959371 (best action = 6568) 
list of simulation results
f(x)=-0.959371 (action = 6568)
f(x)=-0.990530 (action = 5365)
f(x)=-0.996815 (action = 8990)
f(x)=-1.118381 (action = 2987)
f(x)=-1.0